In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from config import DOWNLOAD_DIR, INPUT_DIR
from pathlib import Path

## Perform indicies calculations

 * SPI - standardized precipitation index
 * SPEI (can ERA 5 do this?) – standardized potential evaporation index
 * Total Precip
 * % of Normal Precipitation
 * SWE - snow water equivalent
 * SWE % of Normal 
 * Soil Moisture Deficit
